# Verifying the hemispheric ordering of the Glasser parcels and network partitions

#### Taku Ito
#### 02/26/2017

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import statsmodels.sandbox.stats.multicomp as mc
import os
import nibabel as nib
from sklearn.svm import SVC
import utils

In [2]:
def convertCSVToCIFTI64k(inputfilename,outputfilename):
#     basedir = '/projects/AnalysisTools/ParcelsGlasser2016/'
    ciftitemplate = '/projects3/StroopActFlow/data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii'
#     ciftitemplate = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
    wb_command = 'wb_command -cifti-convert -from-text' 
    wb_command += ' ' + inputfilename 
    wb_command += ' ' + ciftitemplate
    wb_command += ' ' + outputfilename
    wb_command += " -col-delim ','"
    wb_command += ' -reset-scalars'
    os.system(wb_command)
#     print wb_command

In [16]:
def convertCSVToCIFTI64k2(inputfilename,outputfilename):
#     basedir = '/projects/AnalysisTools/ParcelsGlasser2016/'
    ciftitemplate = '/projects3/StroopActFlow/data/Q1-Q6_RelatedParcellation210.RL.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
#     ciftitemplate = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
    wb_command = 'wb_command -cifti-convert -from-text' 
    wb_command += ' ' + inputfilename 
    wb_command += ' ' + ciftitemplate
    wb_command += ' ' + outputfilename
    wb_command += " -col-delim ','"
    wb_command += ' -reset-scalars'
    os.system(wb_command)

In [19]:
subjNums = ['101', '102']

basedir = '/projects3/StroopActFlow/data/'
resultsdir = basedir + 'results/'
restdir = resultsdir + 'glmRest_GlasserParcels/'

glasser_nets = np.loadtxt(basedir + 'NetworkPartitions3.1/indexsort.txt', delimiter=',')
# Make into python numbering (starts from 0)
glasser_nets -= 1.0
networkorder = glasser_nets.astype(int)
networkorder.shape = (len(networkorder),1)

networkmappings = {'fpn':6, 'vis':1, 'smn':2, 'con':3, 'dmn':8, 'aud':7, 'van':5, 'dan':4}
networks = networkmappings.keys()

networkdef = basedir + 'NetworkPartitions3.1/network_partition.txt'
networkdef = np.loadtxt(networkdef, delimiter=',')
xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net
    
# Load in Glasser parcels
glasserfile = '/projects3/StroopActFlow/data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii'
glasser = nib.load(glasserfile).get_data()
glasser = np.squeeze(glasser)

# Load in Glasser parcels2
glasserfile = '/projects3/StroopActFlow/data/Q1-Q6_RelatedParcellation210.RL.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile).get_data()
glasser2 = np.squeeze(glasser2)

## Visualize VAN when parcels are ordered from Left -> Right

In [30]:
glasser2[0:100]

memmap([ 215.,  219.,  192.,  206.,  193.,  329.,  265.,  340.,  313.,
        282.,  311.,  214.,  214.,  214.,  214.,  194.,  194.,  194.,
        194.,  194.,  214.,  214.,  214.,  214.,  214.,  214.,  213.,
        213.,  213.,  213.,  213.,  213.,  213.,  213.,  213.,  213.,
        211.,  211.,  211.,  211.,  211.,  211.,  211.,  211.,  211.,
        301.,  301.,  301.,  301.,  301.,  301.,  301.,  301.,  301.,
        181.,  181.,  181.,  181.,  181.,  181.,  181.,  181.,  181.,
        184.,  184.,  184.,  184.,  333.,  333.,  333.,  333.,  333.,
        333.,  333.,  333.,  333.,  300.,  300.,  300.,  299.,  299.,
        299.,  299.,  299.,  306.,  306.,  306.,  306.,  306.,  306.,
        333.,  333.,  333.,  333.,  333.,  214.,  214.,  214.,  194.,  194.], dtype=float32)

In [37]:
glasser[180:]

memmap([ 213.,  194.,  194., ...,  133.,  133.,  133.], dtype=float32)

In [33]:
networkdef2 = np.zeros((networkdef.shape))
networkdef2[0:180] = networkdef[180:]
networkdef2[180:] = networkdef[0:180]
# networkdef = networkdef2

In [25]:
van_ind = np.where(networkdef2==networkmappings['van'])[0]

v_ind = []
for roi in van_ind:
    vertices = np.where(glasser==roi+1)[0]
    v_ind.extend(vertices)
    
v_ind = np.asarray(v_ind)

van_left2right = np.zeros((glasser.shape[0],))
van_left2right[v_ind] = 1.0

dire = '/projects3/StroopActFlow/data/results/GlasserNetworkHemisphereValidation/'
filename = dire+'netpartitions_van_left2right'
np.savetxt(filename+'.csv', van_left2right, delimiter=',')

convertCSVToCIFTI64k(filename+'.csv',filename+'.dscalar.nii')

## Visualize VAN when parcels are ordered from Right -> Left

In [26]:
van_ind = np.where(networkdef==networkmappings['van'])[0]

v_ind = []
for roi in van_ind:
    vertices = np.where(glasser2==roi+1)[0]
    v_ind.extend(vertices)
    
v_ind = np.asarray(v_ind)

van_right2left = np.zeros((glasser2.shape[0],))
van_right2left[v_ind] = 1.0

dire = '/projects3/StroopActFlow/data/results/GlasserNetworkHemisphereValidation/'
filename = dire+'netpartitions_van_right2left'
np.savetxt(filename+'.csv', van_right2left, delimiter=',')

convertCSVToCIFTI64k2(filename+'.csv',filename+'.dscalar.nii')